In [1]:
# dependencies
import torch
import numpy as np
from torchvision.models import resnet18
import torch.optim as optim
from torch import nn
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

#################### Your Code ####################
ROOT_PATH=globals()['_dh'][0]  # Jupyter Notebook hack to get location of notebook file, https://stackoverflow.com/a/53958599
###################################################

BATCH_SIZE=1024

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = CIFAR10(root=ROOT_PATH, download=True, train=True, transform=transform)
eval_dataset = CIFAR10(root=ROOT_PATH, train=False, transform=transform)

train_data_loader = DataLoader(dataset=train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
eval_data_loader = DataLoader(dataset=eval_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=False)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = resnet18(weights = None, num_classes = 10)

Files already downloaded and verified


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [21]:
correct = 0
model.load_state_dict(torch.load('model_weights.pth'))
model.to(device)
model.eval()

with torch.no_grad():
    for batch in eval_data_loader:
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        out = model(images)
        out = out.argmax(dim = 1)
        # results = out - labels
        # results = results.cpu()
        # correct += np.count_nonzero(results==0)
        correct += (out == labels).sum().cpu().item()
    print(correct, len(eval_dataset), correct/len(eval_dataset))
    

5429 10000 0.5429


In [20]:
(out[:10] == labels[:10]).sum()

tensor(7, device='cuda:0')

In [19]:
labels[:10]

tensor([6, 9, 3, 9, 8, 7, 7, 1, 6, 5], device='cuda:0')